In [2]:
import csv
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re

In [4]:
def user_scrape(username):
    #  build user url
    url = f"https://www.beeradvocate.com/user/beers/?ba={username}&order=ratingD&start=0&view=R"
    
    #  make request 
    response = requests.get(url)
    
    # beatiful soup object
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # grab user info table
    profile_table = soup.find("div", class_ = "pairsJustified")
    
    # data-points
    dls = profile_table.find_all('dl')
    
    #  dictionary holding user states
    profile_dict = {"user_name" : username}
    
    # loop through table items and make dictionary keys and values
    for dl in dls:
        profile_dict[dl.find('dt').text] = dl.find('dd').text
    
    # get the ratings table on the page
    table = soup.find('table')
    # get the ratings by row
    user_ratings = table.find_all('tr')[3:]
    
    # list for top 3
    top_3 = []

    beer_links = []
    for x in range(3):
        text_info = user_ratings[x].find_all('a')
        num_info = user_ratings[x].find_all('td')[-3:-1]
        base_url_ba = "https://www.beeradvocate.com"
        beer_link = base_url_ba + text_info[0]['href'][0:text_info[0]['href'].find("?")]
        beer_links.append(beer_link)
        top_3_dict = {"Beer Name": text_info[0].text,
                     "Brewery": text_info[1].text,
                     "Beer Style": text_info[2].text,
                     "ABV": num_info[0].text,
                     "User Rating": num_info[1].text}
        top_3.append(top_3_dict)
        
    for link in beer_links:
        response = requests.get(link)
        soup = BeautifulSoup(response.text, 'html.parser')
        beerimages = soup.find('div', {'id': 'main_pic_norm'}).find_all('img')
        beerlink = beerimages[0]['src']
        if (beerlink is not None):
            top_3_dict['image_link'] = beerlink
        else:
            top_3_dict['image_link'] = ""

    return profile_dict, top_3, beer_links
    

profile, top_3, beer_links = user_scrape("brutaltruth")


In [5]:
profile

{'Beer Styles:': '92',
 'Beers Added / Pics Uploaded:': '100 / 0',
 'Beers Rated / Reviewed:': '1,656 / 1,653',
 'Brewers:': '441',
 'US States / Countries:': '40 / 30',
 'user_name': 'brutaltruth'}

In [6]:
top_3

[{'ABV': '16.90',
  'Beer Name': 'Rumpkin',
  'Beer Style': 'Pumpkin Ale',
  'Brewery': 'Avery Brewing Company',
  'User Rating': '4.74'},
 {'ABV': '11.50',
  'Beer Name': 'Imperial Doughnut Break',
  'Beer Style': 'American Porter',
  'Brewery': 'Evil Twin Brewing',
  'User Rating': '4.63'},
 {'ABV': '10.50',
  'Beer Name': 'Founders Imperial Stout',
  'Beer Style': 'Russian Imperial Stout',
  'Brewery': 'Founders Brewing Company',
  'User Rating': '4.6',
  'image_link': 'https://cdn.beeradvocate.com/im/beers/21822.jpg'}]

In [8]:
beer_links

['https://www.beeradvocate.com/beer/profile/30/62449/',
 'https://www.beeradvocate.com/beer/profile/24300/90473/',
 'https://www.beeradvocate.com/beer/profile/1199/21822/']